# 线性回归的简洁实现

In [1]:
import torch
import numpy as np
from torch.utils import data
from d2l import torch as d2l

# 1. 生成数据集

In [2]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

# 2. 读取数据集

In [4]:
# TensorDataset 可以用来对 tensor 进行打包，就好像 python 中的 zip 功能。
# 该类通过每一个 tensor 的第一个维度进行索引。因此，该类中的 tensor 第一维度必须相等

a = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9], [1, 2, 3], [4, 5, 6], [7, 8, 9], 
                  [1, 2, 3], [4, 5, 6], [7, 8, 9], [1, 2, 3], [4, 5, 6], [7, 8, 9]])

b = torch.tensor([44, 55, 66, 44, 55, 66, 44, 55, 66, 44, 55, 66])

train_ids = data.TensorDataset(a, b) #相当于zip函数

In [10]:
for x, y in train_ids:
    print(x, "\t", y)

tensor([1, 2, 3]) 	 tensor(44)
tensor([4, 5, 6]) 	 tensor(55)
tensor([7, 8, 9]) 	 tensor(66)
tensor([1, 2, 3]) 	 tensor(44)
tensor([4, 5, 6]) 	 tensor(55)
tensor([7, 8, 9]) 	 tensor(66)
tensor([1, 2, 3]) 	 tensor(44)
tensor([4, 5, 6]) 	 tensor(55)
tensor([7, 8, 9]) 	 tensor(66)
tensor([1, 2, 3]) 	 tensor(44)
tensor([4, 5, 6]) 	 tensor(55)
tensor([7, 8, 9]) 	 tensor(66)


In [11]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [12]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [13]:
# 读取并打印第一个小批量样本
# 这里我们使用 iter 来构造Python迭代器，并使用next()从迭代器中获取第一项

next((iter(data_iter)))

[tensor([[ 2.5866, -1.0703],
         [ 0.9033, -0.9955],
         [ 1.6367,  0.7435],
         [ 0.5530, -1.4914],
         [-1.2659,  0.1651],
         [ 0.6968,  2.6223],
         [-0.2903, -0.2618],
         [-0.7968, -0.6303],
         [ 0.6292,  0.1415],
         [-1.2684, -0.2513]]),
 tensor([[12.9994],
         [ 9.3823],
         [ 4.9494],
         [10.3625],
         [ 1.0872],
         [-3.3172],
         [ 4.5143],
         [ 4.7385],
         [ 4.9802],
         [ 2.5486]])]

# 3. 定义模型

In [14]:
# 我们首先定义一个模型变量 net，它是一个Sequential类的实例
# 然后在这个实例中添加层的实例
# 全连接层是在 Linear 类中定义的

from torch import nn

net = nn.Sequential(nn.Linear(in_features=2, out_features=1, bias=True))

In [23]:
# 显示模型待训练的参数
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.0149, 0.0061]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


# 4. 初始化模型参数

In [17]:
# 通过 net[0] 来选择网络中的第一个图层，然后使用 weight.data 和 bias.data 方法来访问参数
# 现在是系统自动为我们做的初始化
net[0].weight.data, net[0].bias.data

(tensor([[ 0.0854, -0.3812]]), tensor([-0.1068]))

In [48]:
# 对权重矩阵和偏置，我们进一步使用方法来初始化
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [49]:
# 参数初始化完毕
net[0].weight.data, net[0].bias.data

(tensor([[ 0.0050, -0.0032]]), tensor([0.]))

# 5. 定义损失函数

In [50]:
# 计算均方误差使用的是 MSELoss类，默认情况下，返回的是所有样本损失的平均值

lossMSE = nn.MSELoss()

# Huber损失函数
lossHuber = nn.HuberLoss(delta=1)

# 6. 定义优化算法

In [51]:
# 小批量随机梯度下降法是一种优化神经网络的标准工具
# pytorch在 optim 模块中实现了该算法的许多变种
# 使用的方法就是创建一个对应的实例，然后指定优化的参数以及其他的超参数，可以以关键字传入，也可以通过字典传入

trainer = torch.optim.SGD(net.parameters(), lr=0.03)

# 7. 训练

在每个迭代周期里（epoch），我们将完整的遍历一遍数据集，不停地从中获取一个小批量的输入和相应的标签。

对于每一个小批量，我们进行如下步骤：
1. 通过调用`net(x)` 生成预测并计算损失`l`（前向传播）
2. 通过反向传播来计算梯度
3. 通过调用优化器来更新模型参数
4. 计算每个迭代周期（epoch）后的损失，用来监控训练过程

In [52]:
num_epochs = 5

for epoch in range(num_epochs):
    for X, y in data_iter:
        trainer.zero_grad()
        y_pred = net(X)
        # l = lossMSE(y_pred, y)
        l = lossHuber(y_pred, y)       
        l.backward()
        trainer.step()
        
    # l = lossMSE(net(features), labels)
    l = lossHuber(net(features), labels)
    print(f"epoch {epoch + 1}, loss {l:f}")

epoch 1, loss 2.273136
epoch 2, loss 0.311876
epoch 3, loss 0.000669
epoch 4, loss 0.000049
epoch 5, loss 0.000047


In [53]:
net[0].weight.data, net[0].bias.data

(tensor([[ 2.0012, -3.3995]]), tensor([4.1999]))

In [54]:
# 获取训练后的参数
w = net[0].weight.data
b = net[0].bias.data

print(f"w的估计误差为：{true_w - w.reshape(true_w.shape)}")
print(f"b的估计误差为：{true_b - b}")

w的估计误差为：tensor([-0.0012, -0.0005])
b的估计误差为：tensor([0.0001])


# 8. 训练的基本框架

In [ ]:
# 下面便是训练模型的通用框架

for input, target in dataset:
    optimizer.zero_grad()
    output = model(input)
    loss = loss_fn(output, target)
    loss.backward()
    optimizer.step()